### Some common terms

  >Agent
  
  >Environment
  
  >Actions,Rewards,Obseravtions

In [1]:
!pip install gym

You should consider upgrading via the 'c:\users\admin\anaconda3\python.exe -m pip install --upgrade pip' command.


### Interacting with gym API

In [41]:
import gym 

In [42]:
#Create environment
env = gym.make('CartPole-v0')



### Certain methods and attributes

- action_space

- obseravtions_space

- reset() : returns initials state and resets the environment

- step()

- render()

In [43]:
env.reset()#gives environment initial state

array([-0.01066226,  0.0377022 , -0.04893415,  0.0112236 ])

In [44]:
env.action_space

Discrete(2)

In [45]:
env.action_space.n

2

In [46]:
env.observation_space#Box class is a class which represent n dimenional tensor

Box(4,)

In [47]:
env.reset()
for i in range(1000):
    random_action = env.action_space.sample() # take a random action
    env.step(random_action)#randomly move to left or right
    env.render()
    
env.close()

C:\Users\admin\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [48]:
#max reward = 200(win)


### Playing Game with a random strategy

- Game Episode

- Step() Function

- Game over?

In [49]:
for e in range(20):#e = episodes
    #play 20 games from satrt to end
    observation = env.reset()
    for t in range(50):
        env.render()
        action = env.action_space.sample()
        obs,reward,done,other_info = env.step(action)
        if done:
            #game episode is over
            print("Game Episode: {}/{} High Score:{}".format(e,20,t))
            break
env.close()
print("All 20 episodes over")

Game Episode: 0/20 High Score:19
Game Episode: 1/20 High Score:10
Game Episode: 2/20 High Score:21
Game Episode: 3/20 High Score:13
Game Episode: 4/20 High Score:18
Game Episode: 5/20 High Score:10
Game Episode: 6/20 High Score:12
Game Episode: 7/20 High Score:14
Game Episode: 9/20 High Score:12
Game Episode: 10/20 High Score:14
Game Episode: 11/20 High Score:31
Game Episode: 12/20 High Score:23
Game Episode: 13/20 High Score:19
Game Episode: 14/20 High Score:8
Game Episode: 15/20 High Score:33
Game Episode: 16/20 High Score:21
Game Episode: 17/20 High Score:11
Game Episode: 18/20 High Score:15
Game Episode: 19/20 High Score:14
All 20 episodes over


### Q Learning

### Agent design and Neural Network

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random

### Training the DQN agent(DeeoQ-learner)

In [55]:
n_episodes= 1000
out_dir = "Untitled Folder 2/"


In [56]:
agent = Agent(state_size=4,action_size=2)
done = False
state_size=4
action_size=2

In [57]:
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    for time in range(500):
        env.render(close=True)
        action = agent.act(state)
        next_state,reward,done,other_info = env.step(action)
        reward = reward if not done else -10
        next_state= np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)#Exp for the agent
        if done:
            print("Game Episode: {}/{} High Score:{} Exploration Rate:{:.2f}".format(e,n_episodes,time,agent.epsilon))
            break
    if len(agent.memory)>32:
        agent.train(32)
    if e%50 == 0:
        agent.save(out_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
print("Model trained")  
env.close()

TypeError: render() got an unexpected keyword argument 'close'

In [36]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)

In [37]:
n_episodes= 500
out_dir = "Untitled Folder 2/"


In [38]:
agent = Agent(state_size=4,action_size=2)
done = False
state_size=4
action_size=2

In [40]:
agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>32:
        agent.train(32)
    
    if e%50==0:
        agent.save(out_dir+"weights_"+'{:04d}'.format(e)+".hdf5")
        
env.close()

Game Episode :0/500, High Score:17,Exploration Rate:1.0
Game Episode :1/500, High Score:10,Exploration Rate:1.0
Game Episode :2/500, High Score:16,Exploration Rate:1.0
Game Episode :3/500, High Score:34,Exploration Rate:0.99
Game Episode :4/500, High Score:19,Exploration Rate:0.99
Game Episode :5/500, High Score:23,Exploration Rate:0.99
Game Episode :6/500, High Score:20,Exploration Rate:0.98
Game Episode :7/500, High Score:10,Exploration Rate:0.98
Game Episode :8/500, High Score:17,Exploration Rate:0.97
Game Episode :9/500, High Score:14,Exploration Rate:0.97
Game Episode :10/500, High Score:11,Exploration Rate:0.96
Game Episode :11/500, High Score:34,Exploration Rate:0.96
Game Episode :12/500, High Score:22,Exploration Rate:0.95
Game Episode :13/500, High Score:15,Exploration Rate:0.95
Game Episode :14/500, High Score:18,Exploration Rate:0.94
Game Episode :15/500, High Score:29,Exploration Rate:0.94
Game Episode :16/500, High Score:10,Exploration Rate:0.93
Game Episode :17/500, High 

Game Episode :142/500, High Score:30,Exploration Rate:0.5
Game Episode :143/500, High Score:13,Exploration Rate:0.49
Game Episode :144/500, High Score:30,Exploration Rate:0.49
Game Episode :145/500, High Score:33,Exploration Rate:0.49
Game Episode :146/500, High Score:42,Exploration Rate:0.49
Game Episode :147/500, High Score:69,Exploration Rate:0.48
Game Episode :148/500, High Score:35,Exploration Rate:0.48
Game Episode :149/500, High Score:15,Exploration Rate:0.48
Game Episode :150/500, High Score:53,Exploration Rate:0.48
Game Episode :151/500, High Score:30,Exploration Rate:0.47
Game Episode :152/500, High Score:82,Exploration Rate:0.47
Game Episode :153/500, High Score:61,Exploration Rate:0.47
Game Episode :154/500, High Score:32,Exploration Rate:0.47
Game Episode :155/500, High Score:58,Exploration Rate:0.46
Game Episode :156/500, High Score:31,Exploration Rate:0.46
Game Episode :157/500, High Score:30,Exploration Rate:0.46
Game Episode :158/500, High Score:16,Exploration Rate:0.4

KeyboardInterrupt: 